In [32]:
import pandas as pd
import numpy as np


import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation

file = "C:/Users/TYS/Documents/study/Master/INSY662/group project/Real Code/INSY662-Group-Project/Data/complete_dataset_full.csv"
df = pd.read_csv(file)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/TYS/Documents/study/Master/INSY662/group project/Real Code/INSY662-Group-Project/Data/complete_dataset.csv'

In [ ]:
df.head

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

# Identify numerical columns (assuming they are the ones with type 'float64')
numerical_cols = df.select_dtypes(include=['float64']).columns.tolist()

# Define our features and target
X = df.drop('rideable_type_electric_bike', axis=1)
y = df['rideable_type_electric_bike']

# Standardize the numerical features
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])



In [ ]:
# Initialize the logistic regression model with L1 penalty, which is Lasso Regularization
logistic = LogisticRegression(penalty='l1', solver='liblinear', random_state=0)

# Feature selection using SelectFromModel
selector = SelectFromModel(estimator=logistic)
X_new = selector.fit_transform(X, y)

# Get the support mask to understand which features are selected
selected_features_mask = selector.get_support()
selected_features = X.columns[selected_features_mask]

selected_features.tolist()

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# First, we'll create a new DataFrame with only the selected features
X_selected = X[selected_features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 5)

lr = LogisticRegression(max_iter=1000) # automatically runs a regularized logistic regression; to control the penalty, add argument 'C = value'; C represents the inverse of lambda
lr_model = lr.fit(X_train,y_train) 

y_pred = lr.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Generate a classification report
class_report = classification_report(y_test, y_pred)

accuracy, class_report

0.6945454545454546

For class 0 (not electric bike):
Precision: 74% (of all instances predicted as class 0, 74% were actually class 0)
Recall: 59% (of all actual instances of class 0, 59% were correctly predicted)
F1-score: 65% (a weighted average of precision and recall for class 0)


For class 1 (electric bike):
Precision: 68% (of all instances predicted as class 1, 68% were actually class 1)
Recall: 81% (of all actual instances of class 1, 81% were correctly predicted)
F1-score: 74% (a weighted average of precision and recall for class 1)

In [ ]:
from sklearn.decomposition import PCA

# Initialize PCA, we'll choose to keep enough components to explain 95% of the variance
pca = PCA(n_components=0.95, random_state=0)

# Fit PCA on the features
pca.fit(X)

# Transform the features using PCA
X_pca = pca.transform(X)

# Number of components
n_components = pca.n_components_

# Explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_.cumsum()

n_components, explained_variance_ratio